# Assignment 4: Explainability

*Part of the course:
Machine Learning (code: INFOB3ML), fall 2022, Utrecht University*

Total points: 10 (100%)

Deadline: Friday 4 November, 23:59

**Write your names and student numbers here: ___**

Submit one ipynb file per pair.

**Before you submit, click Kernel > Restart & Run All to make sure you submit a working version of your code!**



## Installation

For this assignment, we are going to use the following Python packages:

matplotlib, pandas, statsmodels, interpret, scikit-learn, openpyxl and graphviz

In [33]:
# Installing packages
#!conda install python-graphviz
%conda install python-graphviz
#!pip install matplotlib pandas statsmodels interpret sklearn openpyxl
# Needed for PDP
#!pip install scikit-learn --upgrade

^C

Note: you may need to restart the kernel to use updated packages.


## Downloading the data
We are going to use the combined cycle power plant dataset. This dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. We have the following features: hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V). We will train ML models to predict the net hourly electrical energy output (EP) of the plant.

For a detailed description, see: [[Description](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant)]

We first need to download and prepare data. 


In [ ]:
# Download and unzip data
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
!unzip CCPP.zip

## Loading and preprocessing the data
We split the data into training (first 5000 instances) and validation (the subsequent 2000) and test (the last 2568) sets. We will use the training set to train a model, and validation set to optimize the model hyper-parameters. 


In [1]:
# Load and prepare data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# global variables
DATA_FILENAME = 'CCPP/Folds5x2_pp.xlsx'
FEATURE_NAMES = ['AT', 'V', 'AP', 'RH']
LABEL_NAME = 'PE'
# Load the data from the excel file
def load_data():
    def split_feature_label(data_set):
        features = data_set[FEATURE_NAMES]
        labels = data_set[LABEL_NAME]
        return features, labels

    data = pd.read_excel(DATA_FILENAME)
    train_set, dev_set, test_set = data[:5000], data[5000: 7000], data[7000:]

    train_features, train_labels = split_feature_label(train_set)
    dev_features, dev_labels = split_feature_label(dev_set)
    test_features, test_labels = split_feature_label(test_set)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels


# preprocess (by z-normalization) the data for the regression task
# return the normalized feature sets and corresponding target variables 
def prepare_load_regression_data():
    train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels = load_data()

    scaler = StandardScaler()
    scaler = scaler.fit(train_features)
    train_features = pd.DataFrame(data=scaler.transform(train_features), columns=FEATURE_NAMES)
    dev_features = pd.DataFrame(data=scaler.transform(dev_features), columns=FEATURE_NAMES)
    test_features = pd.DataFrame(data=scaler.transform(test_features), columns=FEATURE_NAMES)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels



## Training and Interpreting a Linear Regression Model

**Q1**. (10)% Train a linear regression model (we recommend the statsmodels package) and report $R^2$ (goodness of fit) statistic. 

For model interpretability, provide for each feature (+ the bias variable) the following in tabular format: 
* Weight estimates
* SE (standard error of estimates) 
* T statistics 


Further Questions regarding the linear model (answers to be included in the notebook): 

**Q2**. (10%) Which three features are the most important?

**Q3**. (10%) How does the gas turbine energy yield (EP) change with unit (one degree C) increase of the ambient temperature given that all other feature values remain the same? (Note: Here you should consider whether you use the original or z-normalized features to train your linear model.)

**Q4**. (10%) Show bar graph illustrations of the feature effects for the first two validation set instances.

In [9]:
# We recommend the statsmodels package
import statsmodels.api as sm
# Hint, by default this sm does not include the bias/offset term w_0
# thus, you should add it yourself using sm.add_constant()

# Linear regression

train_features, train_labels, dev_features, dev_labels, test_features, test_labels = prepare_load_regression_data()

X = None
X = sm.add_constant(train_features)

model = sm.OLS(train_labels, X)
results = model.fit()
print(results.summary())
print("R^2: ", results.rsquared)
print("T-values:\n ", results.tvalues)

                            OLS Regression Results                            
Dep. Variable:                     PE   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                 1.644e+04
Date:                Tue, 25 Oct 2022   Prob (F-statistic):               0.00
Time:                        18:37:57   Log-Likelihood:                -14635.
No. Observations:                5000   AIC:                         2.928e+04
Df Residuals:                    4995   BIC:                         2.931e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        454.4443      0.064   7108.416      0.0

C:\Users\31617\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



Reflection: why would training a regression tree not work well for this dataset in terms of model interpretability? (answer in the notebook)

## Training and Interpreting an Explainable Boosting Model (Generalized Additive Model)
**Q6**. (20%) Train a Explainable Boosting Machine (with [interpret.ml](https://github.com/interpretml/interpret/))

(Note on grading: Training EBM 10%, global and local explanation visualizations - see below -  5% each)

For a tutorial see: [[Tutorial](https://nbviewer.org/github/interpretml/interpret/blob/master/examples/python/notebooks/Interpretable%20Regression%20Methods.ipynb)]


* (5%) Visualize/provide global (model-wise) feature importances for EBM as a table or figure. What are the most important two features in EBM? Are they the same as in the linear model? 
* (5%) Visualize local (instance-wise) feature importances for a development set instance of your choice.


In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor

# EBM

## Training and Explaining Neural Networks
**Q7**. (20%) Train a Neural Network (using the training and validation sets): One-layer MLP (ReLU activation function + 50 hidden neurons) 

We recommend to use the Adam optimizer. Fine-tune the learning rate and any other hyper-parameters you find necessary. 

For a tutorial see: [[Tutorial](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)]

Your code should report the results following the instructions below:

Note on grading: training NN: 8%, answering below sub-questions 4% each.

* (4%) Apply the trained neural network model on the test set and report Root Mean Square Error (RMSE) performance measure.

* Analyzing factors influencing the neural network predictions. 
See the [Documentation](https://scikit-learn.org/stable/modules/partial_dependence.html) to use Partial Dependence Plot (PDP) and Independent Conditional Expectation (ICE) implementations in python.
Use the trained one-layer MLP model to
  * (4%) Generate and report a bivariate PDP using 'AT' (Ambient Temperature) and 'V' (Exhaust Vacuum) features (Note: not two univariate PDPs but one bivariate PDP).
  * (4%) Generate  ICE plots for each feature.

In [ ]:
from sklearn.neural_network import MLPRegressor

# One-layer MLP use  learning_rate_init=0.001 to get a reasonable model, optimize other parameters by experimentation
# an RMSE around 4.2 is reasonable


### Generating Model-Agnostic Explanations for NN predictions
You can check the tutorials for LIME explanations for neural networks 
[[LIME Tutorial](https://nbviewer.org/github/interpretml/interpret/blob/master/examples/python/notebooks/Explaining%20Blackbox%20Regressors.ipynb)]


**Q8**. (10%) Provide explanations for two randomly selected test set instances using LIME for the trained NN model. 

In [ ]:
# Global explanations
import graphviz
from interpret import show

# Local explanations (LIME)
from interpret.blackbox import LimeTabular
